# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_mixed")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

Using framework PyTorch: 1.10.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
/usr/local/lib/python3.8/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:559: TracerWarning: Converting a tensor to a Python boolean might 

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cuda:0


In [6]:
prompt = "In my dreams, I'm a dragon"
for i in range(2):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon, and I'm a dragoness. I'm a dragoness, and I'm a dragoness, but I'm not sure what exactly I'll do. I'll just show you some of the fun I'll have."<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon. I can't stand or walk like a human, but I can certainly see that."I'm not a monster in the way you would think I'm a monster."<|endoftext|>
PyTorch: In my dreams, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon, I'm a dragon
----------------------------------------------------------------------------------------------------
ONNX: In my dreams, I'm a dragon, and I'm a dragoness. I'm a dragoness, and I'm a dragoness. I'm a dragoness, and I'm a dragoness. I'm a dragoness, a

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [7]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hey!"', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "Well, I'm not sure if there is anything that can fill the void."<p><msg>c "I think I'll have to see if I can make it up."<d><scn>park2<msg>Ry "I think I'll have to see if I can make it up."<|endoftext|>

[ONNX] Reply: park2<msg>Ry "Well, I'm not sure if there is anything that can fill the void."<p><msg>c "I think I'll have to see if I can make it up."<d><scn>park2<msg>Ry "I think I'll have to see if I can make it up."<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ry "I'm not sure. I'll have to see if you can keep it up."<p><msg>c "I'll see myself out."<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hey."<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ry "I think he was right to think that I should consider him a friend."<p><msg>c "I was with him for a long time, but I think he was right to think that I should consider him a friend."<p><msg>c "I was with him for a long time, but I think he was righ

# Sampling test

This is gonna be interesting!

In [8]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "Well, I'm not sure if there is anything special about it."<d><scn>park2<msg>Ry "I'm not sure about that. I suppose it's just a souvenir for when I get back home."<p><msg>c "I'll keep that in mind."<|endoftext|>

[ONNX] Reply: park2<msg>Ry "Well."<p><msg>c "Hey, what's up?"<d><scn>park2<msg>Ry "I'm having a drinking contest with a few people. They think I'm going to be the one that's supposed."<d><scn>park2<msg>Ry "I think I'll see if I can make it. I'll be right back."<d><scn>park2<msg>Ry "I'm not so sure about it. I suppose you have your own opinion."<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ry "I'm not sure, but you know what? It doesn't matter what you thought of it. You're right."<p><msg>c "I'm not surprised. I'm not surprised by the fact."<d><scn>park2<msg>Ry "You know what, maybe you should do something else."<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Oh yeah, I do have a few steaks I could give you."<|endoftext|>

------

# RP test
Testing out the injected roleplay actions

In [9]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Oh, [player_name], I wasn't expecting visitors."<|endoftext|>
[ONNX] Visit Lorem -> loremapt<msg>Lo "Oh, [Where is it?"<|endoftext|>
[ONNX Quantized] Visit Lorem -> loremapt<msg>Lo "Oh. I see you today. You know, we're already here."<d><scn>loremapt<msg>Lo "I'm afraid you'll be stuck with us a little longer than you're here, and you know what? It's too much to be that hard."<p><msg>c "I'll see myself out, we'll be back in about a day."<d><scn>loremapt<msg>Lo "I'm not sure if that's what I'm talking about right now. I'll see you next time."<|endoftext|>
----------
[Pytorch] Meet with Lorem -> loremapt<msg>Lo "Oh, [player_name], I wasn't expecting visitors."<|endoftext|>
[ONNX] Meet with Lorem -> loremapt<msg>Lo "Hey [Hey, [Hey, Lore. Are you sure?"<|endoftext|>
[ONNX Quantized] Meet with Lorem -> loremapt<msg>Lo "Oh. Maybe this wasn't such a good idea after all."<|endoftext|>
----------
[Pytorch] Visit Adine -> adineapt<msg>Ad "Oh, [player_name]